In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain
from langchain.chat_models import ChatOpenAI
import os

In [3]:
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [4]:
data = 'reviews.csv'
df = pd.read_csv(data, delimiter=",")
df

,Reviews,Rating,Names
0,"I bought this necklace for my wife's birthday,...",5,Alice
1,This necklace exceeded my expectations. It's d...,5,Bob
2,このネックレスは本当に素晴らしいです。妻へのプレゼントとして購入しましたが、彼女はとても気に...,5,Chisato
3,यह हार बहुत ही सुंदर है। यह मेरे द्वारा खरीदे ...,4,Deepak
4,I'm very pleased with this necklace. It's well...,4,Elenor
5,"The necklace is pretty, but it's more delicate...",3,Francis
6,"The necklace looks good, but the quality is av...",3,George
7,The necklace broke after just one week of wear...,1,Hannah
8,The necklace looks cheap and not as elegant as...,2,Ian
9,期待していたよりも品質が低かったです。チェーンがすぐに切れてしまい、ペンダントもあまり輝きま...,2,Jiro


In [ ]:
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo", verbose=True)

In [6]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [7]:
reply_schema = ResponseSchema(name="Reply",
                             description="Check whether the match is Yes or No.\
                                 If the match is yes, write a suitable response for the review. (The reply should be polite and thankful for positive sentiments and should be apologetic for negative sentiments.)\
                                 If the match is no AND rating is above 3, write a suitable thankful response but sternly mention how the rating does not match the user's sentiment.\
                                 The output should be a single string response in 'Customer's Language' without any mention of match:Yes/No.")
tags_schema = ResponseSchema(name="Tags",
                                      description="Identify the key points mentioned in the review and create a list of those points.\
                                          For example: ['Affordable', 'cheap']. The output should be a single list")

response_schemas = [reply_schema, 
                    tags_schema]

In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
prompt1 = ChatPromptTemplate.from_template('''
    Translate the following review to English:
    \n\n {review}
    '''
)

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key='English_review')

prompt2 = ChatPromptTemplate.from_template('''
    Identify the sentiment of the review and remember it as either POSTIVE, NEUTRAL OR NEGATIVE.
    \n\n Review: {English_review}
    \n\n Check whether the original rating: '{rating}' matches the sentiment you identified (Ratings are from 1-5 and positive sentiment correlates with \
        ratings above 3, neutral for 3 and below 3 is negative sentiment) and remember it as Match: Yes/No
    \n\n Identify the language used here: {review} and remember is as 'Customer's Language'.
    
    Return the following as output:
    {format_instructions}
    '''
)

chain2 = LLMChain(llm=llm, prompt=prompt2, output_key='Result_dictionary')

In [10]:
overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["review", "rating", "format_instructions"],
    output_variables=["Result_dictionary"],
    verbose=False)

In [ ]:
output_df = pd.DataFrame(columns=['Name', 'Reply', 'Tags'])
for i in range(len(df)):
    review = df.Reviews[i]
    rating = df.Rating[i]
    name = df.Names[i]
    chain_output = overall_chain({'review':review, 'rating':rating, 'format_instructions':format_instructions})
    output_dict = output_parser.parse(chain_output.get('Result_dictionary'))
    output_dict['Names'] = name
    new_row = pd.DataFrame([output_dict])
    output_df = pd.concat([output_df, new_row], ignore_index=True)

In [18]:
for i in range(len(df)):
    review = df.loc[i]
    output = output_df.loc[i]
    print(f'''Name: {review.Names}
Review: {review.Reviews}
Rating: {review.Rating}
Reply: {output.Reply}
Tags: {output.Tags}
''')

Name: Alice
Review: I bought this necklace for my wife's birthday, and she absolutely loves it! The craftsmanship is excellent, and the diamonds sparkle beautifully. It's elegant and perfect for both everyday wear and special occasions. Highly recommend!
Rating: 5
Reply: Thank you so much for your kind words! We're thrilled to hear that your wife loves the necklace and that it's perfect for everyday wear and special occasions. We appreciate your recommendation!
Tags: ['Craftsmanship', 'Diamonds', 'Elegant', 'Everyday wear', 'Special occasions']

Name: Bob
Review: This necklace exceeded my expectations. It's dainty but feels substantial and well-made. The pendant catches the light perfectly, making it a real eye-catcher. Great value for the price!
Rating: 5
Reply: Thank you for your glowing review! We are thrilled to hear that the necklace exceeded your expectations and that you find it to be a standout piece. We strive to provide excellent value for our customers, and we are delighted 